# 1.Dataloader与Dataset

Q:如何用torch.utils.data.DataLoader构建可迭代的数据装载器？
- 这是个类：`torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None, generator=None)`
- dataset: Dataset类，决定数据从哪读取及如何读取
- batchsize：批大小
- num_works：是否多进程读取数据
- shuffle：每个epoch是否乱序
- drop_last：当样本数不能被batchsize整除时，是否舍弃最后一批数据

